In [13]:
import tkinter as tk
from pystray import Icon, Menu, MenuItem
from PIL import Image, ImageDraw
import threading
from tkinter import messagebox

In [14]:
import os

# Cài đặt từ requirements.txt
os.system("pip install -r _requirements.txt")

0

In [15]:
LIST_FEATURE_FILES = ['CountTime.py', 'PreventDisplay.py', 'SalaryCal.py', 'ShowMenu.py']

In [16]:
import CountTime, PreventDisplay, SalaryCal, ShowMenu
# Example usage
time_calculator = CountTime.TimeCalculator()
time_calculator.set_start_time(7, 3)
print(time_calculator.cal_end_time(CountTime.FULL_DAY))
print (time_calculator.late)

#==========
# mouse_mover = PreventDisplay.MouseMoverApp()
# mouse_mover.start_moving()
# #================================
# 


Start time set to: 07:03
09:00:00
Not late.
16:51
0:00:00


In [17]:
def submit_time_in(hour: int, minute: int):
    try:
        # Kiểm tra giá trị hợp lệ
        if 0 <= hour < 24 and 0 <= minute < 60:
            time_calculator.set_start_time(hour=hour, minute= minute)
        else:
            messagebox.showerror("Lỗi", "Giờ phải trong khoảng 0-23 và phút trong khoảng 0-59.")
    except ValueError:
        messagebox.showerror("Lỗi", "Vui lòng nhập số hợp lệ.")

def input_time_in():
    # Tạo cửa sổ chính
    inp_time_view = tk.Tk()
    inp_time_view.title("Nhập Giờ và Phút")
    inp_time_view.geometry("200x200")

    # Nhãn và ô nhập giờ
    tk.Label(inp_time_view, text="Giờ:").pack(pady=5)
    hour_entry = tk.Entry(inp_time_view, width=5)
    hour_entry.pack()

    # Nhãn và ô nhập phút
    tk.Label(inp_time_view, text="Phút:").pack(pady=5)
    minute_entry = tk.Entry(inp_time_view, width=5)
    minute_entry.pack()

    # Nút xác nhận với lambda để trì hoãn gọi hàm
    confirm_button = tk.Button(
        inp_time_view, 
        text="OK", 
        command=lambda: submit_time_in(int(hour_entry.get()), int(minute_entry.get()))
    )
    confirm_button.pack(pady=10)
    inp_time_view.mainloop()

def lunch_menu():
    ShowMenu.show_menu()

In [18]:
import tkinter as tk
from datetime import datetime, timedelta

# Danh sách các chế độ và chỉ số hiện tại
modes = ['CẢ NGÀY', 'SÁNG', 'SÁNG TRƯA', 'CHIỀU']
current_mode_index = 0

# Thời gian ra về mẫu cho từng chế độ (giả định)
end_times = {
    'CẢ NGÀY': "12:48",
    'SÁNG': "12:00",
    'SÁNG TRƯA': "12:30",
    'CHIỀU': "12:00"
}

# Lấy giờ ra về từ chế độ hiện tại
mode = modes[current_mode_index]
end_time_str = time_calculator.end_time(current_mode_index)

# Hàm cập nhật chế độ làm việc
def switch_mode():
    global current_mode_index, end_time_str
    current_mode_index = (current_mode_index + 1) % len(modes)
    mode_button.config(text=modes[current_mode_index])
    end_time_str = time_calculator.end_time(current_mode_index)

# Hàm tính và hiển thị thời gian còn lại
def update_remaining_time():
    now = datetime.now()
    
    # Thiết lập thời gian ra về theo giờ phút, giữ nguyên ngày hiện tại
    end_time = datetime.strptime(end_time_str, "%H:%M").replace(
        year=now.year, month=now.month, day=now.day
    )

    # Cập nhật nhãn thời gian
    end_time_label.config(text=f"Giờ ra về: {end_time.strftime('%H:%M')}")
    
    #Tgian còn lại
    if end_time>now:
        remaining_time = end_time - now
        remaining_time_label.config(text=f"Count down: {str(remaining_time).split('.')[0]}")
    else:
        remaining_time = now - end_time
        remaining_time_label.config(text=f"+ {str(remaining_time).split('.')[0]}")

    # Tự động cập nhật mỗi giây
    root.after(1000, update_remaining_time)

# Tạo cửa sổ tkinter
root = tk.Tk()
root.title("Thông tin thời gian làm việc")
root.geometry("250x150")

# Nút chuyển đổi chế độ
mode_button = tk.Button(root, text=modes[current_mode_index], command=switch_mode)
mode_button.pack(pady=10)

# Nhãn hiển thị giờ ra về và thời gian còn lại
end_time_label = tk.Label(root, text="")
end_time_label.pack(pady=5)

remaining_time_label = tk.Label(root, text="")
remaining_time_label.pack(pady=5)

# Bắt đầu cập nhật thời gian
update_remaining_time()

# Chạy vòng lặp giao diện
root.mainloop()

TypeError: 'datetime.time' object is not callable

In [ ]:
class CustomWindow:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("GetTimeIn")
        self.root.geometry("200x150")

        # Sự kiện nhấp đúp chuột để ẩn cửa sổ
        self.root.bind("<Double-Button-1>", self.hide_window)
        # Khởi tạo icon hệ thống
        self.icon = self.create_tray_icon()

        # setup menu
        self.add_menu()

    def add_menu(self):
        # Tạo menu chuột phải
        self.menu = tk.Menu(self.root, tearoff=0)
        self.menu.add_command(label="Input time in", command=input_time_in)
        self.menu.add_command(label="Lunch menu", command=lunch_menu)
        self.menu.add_separator()
        self.menu.add_command(label="Hide", command=self.hide_window)
        self.menu.add_command(label="Exit", command=self.quit_app)
        # Gán sự kiện chuột phải
        self.root.bind("<Button-3>", self.show_context_menu)
    
    def show_context_menu(self, event):
        """Hiển thị menu chuột phải tại vị trí nhấn chuột."""
        try:
            self.menu.tk_popup(event.x_root, event.y_root)
        finally:
            self.menu.grab_release()
        
    def hide_window(self, event=None):
        self.root.withdraw()  # Ẩn cửa sổ

    def show_window(self):
        self.root.deiconify()  # Hiển thị lại cửa sổ

    def quit_app(self):
        self.icon.stop()
        self.root.destroy()

    def create_tray_icon(self):
        # Tạo icon nhỏ cho System Tray
        image = Image.new('RGB', (64, 64), color='blue')
        draw = ImageDraw.Draw(image)
        draw.rectangle((0, 0, 64, 64), fill="blue")

        # Tạo menu cho tray icon
        menu = Menu(
            MenuItem("Show", self.show_window),
            MenuItem("Exit", self.quit_app)
        )

        # Khởi tạo icon với pystray
        icon = Icon("SAT_TimeIn", image, "SAT Time In", menu)
        threading.Thread(target=icon.run, daemon=True).start()
        return icon

    def run(self):
        self.root.mainloop()

    
app = CustomWindow()
app.run()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\SyLV\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\SyLV\AppData\Local\Temp\ipykernel_15624\4251334101.py", line 37, in lunch_menu
    ShowMenu.show_menu()
  File "d:\Work\Auto\GetTimeIn-LO\ShowMenu.py", line 50, in show_menu
    fetch_data(root= root_menu)
  File "d:\Work\Auto\GetTimeIn-LO\ShowMenu.py", line 20, in fetch_data
    display_data(data, root=root)
  File "d:\Work\Auto\GetTimeIn-LO\ShowMenu.py", line 35, in display_data
    img_label = Label(root, image=photo)
                ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\SyLV\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 3248, in __init__
    Widget.__init__(self, master, 'label', cnf, kw)
  File "c:\Users\SyLV\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 2659, in __init__
    self.tk

An error occurred when calling message handler
Traceback (most recent call last):
  File "c:\Users\SyLV\AppData\Local\Programs\Python\Python312\Lib\site-packages\pystray\_win32.py", line 412, in _dispatcher
    return int(icon._message_handlers.get(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\SyLV\AppData\Local\Programs\Python\Python312\Lib\site-packages\pystray\_win32.py", line 224, in _on_notify
    descriptors[index - 1](self)
  File "c:\Users\SyLV\AppData\Local\Programs\Python\Python312\Lib\site-packages\pystray\_base.py", line 328, in inner
    callback(self)
  File "c:\Users\SyLV\AppData\Local\Programs\Python\Python312\Lib\site-packages\pystray\_base.py", line 453, in __call__
    return self._action(icon, self)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\SyLV\AppData\Local\Programs\Python\Python312\Lib\site-packages\pystray\_base.py", line 548, in wrapper0
    return action()
           ^^^^^^^^
  File "C:\Users\SyLV\AppData\Local\Temp\ipykernel_15624\41